Census Record Summary (summarize_csv.py)

In [ ]:
from pathlib import Path
import duckdb
import polars as pl
from tabulate import tabulate
import time

# Define base path for input files
BASE_PATH = Path("D:/source")

Functions

In [ ]:
def create_census_view(con, filename: str):
    csv_path = BASE_PATH / filename
    con.execute(f"""
        CREATE VIEW census AS
        SELECT
            *,
            CAST(cenyear AS TEXT) || '-' || CAST(serial AS TEXT) AS hhid,
            CAST(stateicp AS TEXT) || '-' || CAST(countyicp AS TEXT) AS locid,
            CAST(cenyear AS TEXT) || '-' || CAST(serial AS TEXT) || '-' || CAST(histid AS TEXT) AS pid
        FROM read_csv_auto('{csv_path}', header=true, compression='gzip');
    """)

In [ ]:
def compute_ratios(con):
    query = """
        WITH base AS (
            SELECT hhid, locid, pid FROM census
        ),
        counts AS (
            SELECT
                COUNT(*) AS total_records,
                COUNT(DISTINCT hhid) AS total_hhids,
                COUNT(DISTINCT locid) AS total_locids,
                COUNT(DISTINCT pid) AS total_pids
            FROM base
        ),
        ratios AS (
            SELECT
                total_records,
                total_hhids,
                total_locids,
                total_pids,
                ROUND(CAST(total_pids AS DOUBLE) / total_hhids, 2) AS avg_pids_per_hhid,
                ROUND(CAST(total_hhids AS DOUBLE) / total_locids, 2) AS avg_hhids_per_locid
            FROM counts
        )
        SELECT * FROM ratios;
    """
    return con.execute(query).fetchone()

Summarize the CSV file

In [ ]:
def summarize_csv(filename: str) -> tuple[tuple, float]:
    print(f"Processing: {filename}")  # ✅ appears before progress bar
    con = duckdb.connect(database=':memory:', read_only=False)
    create_census_view(con, filename)

    start = time.perf_counter()
    ratios = compute_ratios(con)
    end = time.perf_counter()

    elapsed = round(end - start, 3)  # seconds, rounded for readability
    return ratios, elapsed

def collect_summaries(filelist: list[str]) -> dict[str, tuple]:
    summary_results = {}
    for filename in filelist:
        ratios, seconds = summarize_csv(filename)
        summary_results[filename] = ratios + (seconds,)  # append time to tuple
    return summary_results

In [ ]:
def create_table(summary_results: dict) -> pl.DataFrame:
    rows = [[fname] + list(vals) for fname, vals in summary_results.items()]
    columns = [
        "filename",
        "records",
        "hhids",
        "locids",
        "pids",
        "pids/hhid",
        "hhids/locid",
        "seconds"  # ⏱️ new column
    ]
    return pl.DataFrame(rows, schema=columns,orient='row')

def print_table(df: pl.DataFrame):
    print("\n" + tabulate(df.rows(), headers=df.columns, tablefmt="github") + "\n")

Execution

In [ ]:
filelist = ["cs1850.csv.gz", "cs1860.csv.gz","cs1870.csv.gz","cs1880.csv.gz","cs1900.csv.gz","cs1910.csv.gz","cs1920.csv.gz"]

# Define output path
SUMMARY_PATH = Path("D:/source/summaries")
SUMMARY_PATH.mkdir(parents=True, exist_ok=True)  # ensure folder exists

summary_results = collect_summaries(filelist)
summary_table = create_table(summary_results)
summary_table.write_csv("summary_table.csv")
print_table(summary_table)

# Write to CSV with prefix
output_filename = SUMMARY_PATH / f"source_table_processing_summary.csv"
summary_table.write_csv(str(output_filename))
